In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest

def load_dataset(filename):
    data = json.load(open(filename, encoding="utf-8"))

    df = pd.json_normalize(
        data,
        record_path="raw_measurements",
        meta=["version","type","start_time","end_time","sampling_rate","weight_filter","zero_offset","expected_weight","robot_type","eoat_params"],
        errors="ignore"
    )

    # Renaming and ordering the data.

    df[["Mx","My","Fz"]] = pd.DataFrame(df.pop("force_vector").tolist(), index=df.index)

    flange = pd.DataFrame(df.pop("robot_tcp.flange").tolist(), index=df.index)
    df[["flange_tx","flange_ty","flange_tz"]] = pd.DataFrame(flange[0].tolist(), index=df.index)
    df[["flange_qw","flange_qx","flange_qy","flange_qz"]] = pd.DataFrame(flange[1].tolist(), index=df.index)

    tcp_off = pd.DataFrame(df.pop("robot_tcp.tcp_offset").tolist(), index=df.index)
    df[["tcp_tx","tcp_ty","tcp_tz"]] = pd.DataFrame(tcp_off[0].tolist(), index=df.index)
    df[["tcp_qw","tcp_qx","tcp_qy","tcp_qz"]] = pd.DataFrame(tcp_off[1].tolist(), index=df.index)

    df[["tcp_vx","tcp_vy","tcp_vz"]] = pd.DataFrame(df.pop("robot_tcp.velocity_linear").tolist(), index=df.index)

    df[["tcp_wx","tcp_wy","tcp_wz"]] = pd.DataFrame(df.pop("robot_tcp.velocity_angular").tolist(), index=df.index)

    df[["joint_base","joint_shoulder","joint_elbow","joint_wrist_1","joint_wrist_2","joint_wrist_3"]] = (
        pd.DataFrame(df.pop("robot_tcp.joint_angles").tolist(), index=df.index)
    )


    t_num = pd.to_numeric(df["timestamp"], errors="coerce")
    t_rel_ms_num = (t_num - t_num.iloc[0]) * 1000
    t_dt = pd.to_datetime(df["timestamp"], errors="coerce")
    t_rel_ms_dt = (t_dt - t_dt.iloc[0]) / pd.to_timedelta(1, "ms")
    time_ms = t_rel_ms_num.fillna(t_rel_ms_dt)


    rt_num = pd.to_numeric(df["robot_tcp.timestamp"], errors="coerce")
    rt_rel_ms = (rt_num - rt_num.iloc[0]) * 1000

    return df

df1 = load_dataset("2025-06-15_11-39-04-387039_measure_weight_FORCE_MEASUREMENT_CASE.json")
df1.head()



,timestamp,value,robot_tcp.timestamp,version,type,start_time,end_time,sampling_rate,weight_filter,zero_offset,...,tcp_vz,tcp_wx,tcp_wy,tcp_wz,joint_base,joint_shoulder,joint_elbow,joint_wrist_1,joint_wrist_2,joint_wrist_3
0,1.749980e+09,-0.054,1.749980e+09,2,measure_weight,1749980344.387157,1749980344.887595,1000.0,"{'filter_name': 'mean', 'kwargs': None}",-0.111579,...,0.888726,0.001071,-0.003127,0.000026,0.240997,-0.769691,-4.098787,-2.847463,5.205006,-1.501638
1,1.749980e+09,-0.055,1.749980e+09,2,measure_weight,1749980344.387157,1749980344.887595,1000.0,"{'filter_name': 'mean', 'kwargs': None}",-0.111579,...,0.888726,0.001071,-0.003127,0.000026,0.240997,-0.769691,-4.098787,-2.847463,5.205006,-1.501638
2,1.749980e+09,-0.055,1.749980e+09,2,measure_weight,1749980344.387157,1749980344.887595,1000.0,"{'filter_name': 'mean', 'kwargs': None}",-0.111579,...,0.888726,0.001071,-0.003127,0.000026,0.240997,-0.769691,-4.098787,-2.847463,5.205006,-1.501638
3,1.749980e+09,-0.055,1.749980e+09,2,measure_weight,1749980344.387157,1749980344.887595,1000.0,"{'filter_name': 'mean', 'kwargs': None}",-0.111579,...,0.888726,0.001071,-0.003127,0.000026,0.240997,-0.769691,-4.098787,-2.847463,5.205006,-1.501638
4,1.749980e+09,-0.055,1.749980e+09,2,measure_weight,1749980344.387157,1749980344.887595,1000.0,"{'filter_name': 'mean', 'kwargs': None}",-0.111579,...,0.888726,0.001071,-0.003127,0.000026,0.240997,-0.769691,-4.098787,-2.847463,5.205006,-1.501638
